In [1]:

import base64
import os
from mistralai import Mistral
from src.utils.config import get_config_value

mistral_token = get_config_value("mistral")
api_key = mistral_token.get("token") 

def encode_image(image_path):
    """Encode the image to base64."""
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file {image_path} was not found.")
        return None
    except Exception as e:  # Added general exception handling
        print(f"Error: {e}")
        return None

def main(): 
    client = Mistral(api_key=api_key)
    ocr_response = client.ocr.process(
        model="mistral-ocr-latest",
        document={
            "type": "image_url",
            "image_url": f"data:image/jpeg;base64,{encode_image("/Users/pawarison/dev/DocuFlow/data/synthetic_data/credit_notes/credit_note_1.png")}" 
        },
        include_image_base64=True
    )
    return ocr_response
    # with open("test.md", "w") as f:
    #     f.write(ocr_response.pages[0].markdown)

if __name__ == "__main__":
    ocr_response = main()

In [27]:
from IPython.display import Markdown, display

display(Markdown(ocr_response.pages[0].markdown))

# บริษัท โกลบอล โลจิสติกส์ โซลูชั่นส์

999/9 อาคารคี ออฟฟิศเคส แอท เซ็นทรัลเวิลด์ ชั้น 40 ถ.พระราม 1 แขวงปทุมวัน เขตปทุมวัน กรุงเทพฯ 10330 โทร: 02-123-4567 เลขประจำตัวผู้เสียภาษี: 0879189666478

ลูกค้า: หจก. สยามการพิมพ์ ที่อยู่: 101 อาคารทรู ศิริทัล พารค ชั้น 7 ถ.สุขุมวิท แขวงบางจาก เขตพระพุทธ กรุงเทพฯ 103984/2025

เรียน: คุณสุนีย์ โทร: 02-123-4567 เลขประจำตัวผู้เสียภาษี: 0560437188559

|  ลำดับ รายการ | จำนวน | ราคา/หน่วย | จำนวนเงิน  |
| --- | --- | --- | --- |
|  1 บริการที่ปรึกษาคำแถวรดลาดศิริทัล (เดือน มี.ย.) | 1 | 30,000.00 | 30,000.00  |
|  2 คาบำรุงรักษาระบบรายปี | 1 | 20,000.00 | 20,000.00  |
|  3 คาเขา Cloud Server - Business Package | 3 | 2,500.00 | 7,500.00  |
|  4 พัฒนาและออกแบบเว็บไซต์ (E-commerce) | 1 | 75,000.00 | 75,000.00  |
|  เรียนโขการข้าระเงิน: ข้าระพันทิเมื่อโต่วันเอกสาร ข้อมูลการโอนเงิน: ธ.กติกริไทย บัญชีออมทรัพย์ 012-3-45678-9 | ยอดรวม (Subtotal) | 132,500.00 |   |
|   | ภาษีมูลค่าเพิ่ม (VAT 7%) | 9,275.00 |   |
|   | ยอดข้าระทั้งสิ้น | 141,775.00 |   |

ผู้อนุมัติ

ผู้รับเงิน / ผู้ลงของ

In [39]:
from openai import OpenAI

system_prompt = (
    "คุณคือผู้เชี่ยวชาญด้านภาษาไทยและเอกสารธุรกิจ "
    "กรุณาตรวจสอบและแก้ไขข้อความด้านล่างนี้ให้ถูกต้องสมบูรณ์ โดย "
    "แก้ไขคำผิด คำตกหล่น หรือข้อความที่อ่านไม่เข้าใจ "
    "แก้ไขคำเฉพาะให้ถูกต้อง เช่น ชื่อบริษัท ชื่อถนน ชื่อแขวง/เขต ชื่อจังหวัด รหัสไปรษณีย์ และข้อมูลที่อยู่ "
    "ให้เป็นมาตรฐานตามข้อมูลจริงในประเทศไทย "
    "หากพบชื่อจังหวัด ชื่อถนน หรือเขต/แขวง ที่สะกดผิดหรือไม่ตรงกับความเป็นจริง ให้แก้ไขให้ถูกต้อง "
    "แก้ไขชื่อธนาคารหรือข้อมูลอื่น ๆ ที่เป็นคำเฉพาะให้ถูกต้อง "
    "แก้ไขเฉพาะข้อความ ไม่ต้องอธิบายการแก้ไข "
    "โปรดแสดงผลลัพธ์เป็นข้อความที่แก้ไขแล้วเท่านั้น"
)

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

completion = client.chat.completions.create(
    model="google/gemma-3n-e2b-it:free",
    messages=[
        {
            "role": "user",
            "content": system_prompt + f"\n\n{ocr_response.pages[0].markdown}"
        }
    ]
)

In [ ]:
print(completion.choices[0].message.content)


บริษัท โกลบอล โลจิสติกส์ โซลูชั่นส์

999/9 อาคารคี ออฟฟิศเคส แอท เซ็นทรัลเวิลด์ ชั้น 40 ถนนพระราม 1 แขวงปทุมวัน เขตปทุมวัน กรุงเทพฯ 10330 โทร: 02-123-4567 เลขประจำตัวผู้เสียภาษี: 0879189666478

ลูกค้า: หจก. สยามการพิมพ์ ที่อยู่: 101 อาคารทรู ศิริทัล พารค ชั้น 7 ถนนสุขุมวิท แขวงบางจาก เขตพระพัฒน์ กรุงเทพฯ 103984/2025

เรียน: คุณสุนีย์ โทร: 02-123-4567 เลขประจำตัวผู้เสียภาษี: 0560437188559

| ลำดับ | รายการ | จำนวน | ราคา/หน่วย | จำนวนเงิน |
|---|---|---|---|---|
| 1 | บริการที่ปรึกษาคำแนะนำการใช้ระบบ (เดือน มี.ย.) | 1 | 30,000.00 | 30,000.00 |
| 2 | ค่าบำรุงรักษาระบบรายปี | 1 | 20,000.00 | 20,000.00 |
| 3 | ค่าเช่า Cloud Server - Business Package | 3 | 2,500.00 | 7,500.00 |
| 4 | พัฒนาและออกแบบเว็บไซต์ (E-commerce) | 1 | 75,000.00 | 75,000.00 |

ยอดรวม (Subtotal) |  |  | 132,500.00 |
| ภาษีมูลค่าเพิ่ม (VAT 7%) |  |  | 9,275.00 |
| ยอดรวมทั้งหมด |  |  | 141,775.00 |

ผู้อนุมัติ

ผู้รับเงิน / ผู้ลงนาม

